In [0]:
import numpy as np
import tensorflow as tf
from numpy import expand_dims
from pickle import dump, load
from keras.models import Model
from readfile import load_vrd_data
from keras.preprocessing import image
from keras.utils import to_categorical
from keras.applications import InceptionV3
from keras.preprocessing.image import img_to_array
from keras.applications.inception_v3 import preprocess_input

In [0]:
#Load data from json files
df_train, df_valid, df_test = load_vrd_data()

In [0]:
#Load the path of images
train_img = []
img_filename = "/content/drive/My Drive/visual_relation/sg_train_images/"
for i in range(len(df_train["source_img"])):
   train_img.append(img_filename + df_train["source_img"][i])
print(len(train_img))

val_img = []
for i in range(len(df_valid["source_img"])):
   val_img.append(img_filename + df_valid["source_img"][i])
print(len(val_img))

img_filename2 = "/content/drive/My Drive/visual_relation/sg_test_images/"
test_img = []
for i in range(len(df_test["source_img"])):
  if df_test["source_img"][i] != "4392556686_44d71ff5a0_o.jpg":
    test_img.append(img_filename2 + df_test["source_img"][i])
print(len(test_img))

In [0]:
train_label = to_categorical(df_train["label"], num_classes=70) 
val_label = to_categorical(df_valid["label"], num_classes=70) 

y_test =[]
for i in range(len(df_test["label"])):
  if df_test["source_img"][i] != "4392556686_44d71ff5a0_o.jpg":
    y_test.append(df_test["label"][i])
test_label = to_categorical(y_test, num_classes=70) 

In [0]:
#Setting up the inception V3 model
original_model    = InceptionV3()
bottleneck_input  = original_model.get_layer(index=0).input
bottleneck_output = original_model.get_layer(index=-2).output
bottleneck_model  = Model(inputs=bottleneck_input, outputs=bottleneck_output)
for layer in bottleneck_model.layers:
    layer.trainable = False

In [0]:
#Extract bounding box information
def bboxes(bbox):
  xmin = bbox[2]
  ymin = bbox[0]
  xmax = bbox[3]
  ymax = bbox[1]
  return xmin,ymin,xmax,ymax

#Preprocess the images before extracting features
def preprocess(img,df_train,i,x):
  img = image.load_img(img)
  xmins,ymins,xmaxs,ymaxs = bboxes(df_train["subject_bbox"][i])
  xmino,ymino,xmaxo,ymaxo = bboxes(df_train["object_bbox"][i])
  if x == "predicate":
    xminp = min(xmins,xmino)
    yminp = min(ymins,ymino)
    xmaxp = max(xmaxs,xmaxo)
    ymaxp = max(ymaxs,ymaxo)
    im_crop = img.crop((xminp,yminp,xmaxp,ymaxp))
  elif x == "subject":
    im_crop = img.crop((xmins,ymins,xmaxs,ymaxs))
  elif x == "object":
    im_crop = img.crop((xmino,ymino,xmaxo,ymaxo))
  
  im = im_crop.resize((299,299))
  im = image.img_to_array(im)
  x = np.expand_dims(im, axis=0)
  x = preprocess_input(x)
  return x

#Extract features
def encode(img,df_train,i,x):
  image = preprocess(img,df_train,i,x)
  fec = bottleneck_model.predict(image)
  return fec

#Create Pickle file
def createfile(l,k):
  pickle_out = open(l,"wb")
  dump(k, pickle_out, protocol=4)
  pickle_out.close()

In [0]:
#Extract features of images
clas = ["predicate","subject","object"]
for z in clas:
  with tf.device('/device:GPU:0'):
    x_val = []
    for i in range(len(val_label)):
      x_val.append(encode(val_img[i],df_valid,i,z))
    
    x_train = []
    for i in range(len(train_label)):
      x_train.append(encode(train_img[i],df_train,i,z))
    
    x_test = []
    for i in range(len(test_label)):
      x_test.append(encode(test_img[i],df_test,i,z))
    
    x_train = np.array(x_train).reshape(-1,2048)
    x_val = np.array(x_val).reshape(-1,2048)
    x_test = np.array(x_test).reshape(-1,2048)

    q,w,e = z + "_train.pickle", z + "_val.pickle", z + "_test.pickle"
    createfile(q,x_train)
    createfile(w,x_val)
    createfile(e,x_test)